In [2]:
import pandas as pd

df1 = pd.read_csv("datasets/Train.csv")
df1.rename(columns={"LONGITUDE":"longitude", "LATITUDE":"latitude"}, inplace=True)
df2 = pd.read_csv("datasets/Centreline Intersection - 4326.csv")


In [3]:
import ast

df2["latitude"]= 0
df2["longitude"] = 0
for i in range(df2.shape[0]):
    df2["longitude"][i] = ast.literal_eval(df2["geometry"][i])['coordinates'][0][0]
    df2["latitude"][i] = ast.literal_eval(df2["geometry"][i])['coordinates'][0][1]

/var/folders/6x/_cgtd_2j2qj3m9pv_1lyx7040000gp/T/ipykernel_16740/2269225457.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["longitude"][i] = ast.literal_eval(df2["geometry"][i])['coordinates'][0][0]
/var/folders/6x/_cgtd_2j2qj3m9pv_1lyx7040000gp/T/ipykernel_16740/2269225457.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-79.5310702158097' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df2["longitude"][i] = ast.literal_eval(df2["geometry"][i])['coordinates'][0][0]
/var/folders/6x/_cgtd_2j2qj3m9pv_1lyx7040000gp/T/ipykernel_16740/2269225457.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [10]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from multiprocessing import Pool, cpu_count

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # radius of Earth in kilometers
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    res = R * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 2)

def calculate_distance(args):
    row_df1, df2 = args
    lon1 = row_df1[-1]["longitude"]
    lat1 = row_df1[-1]["latitude"]
    distances = []
    for row_df2 in df2.iterrows():
        lon2 = row_df2[-1]["longitude"]
        lat2 = row_df2[-1]["latitude"]
        dist = haversine(lat1,lon1,lat2,lon2)
        distances.append(dist)     
    IDX = distances.index(min(distances))
    return df2.iloc[IDX]["INTERSECTION_DESC"]

def get_intersection_based_on_long_lat(df1, df2):
    with Pool(cpu_count()) as p:
        df1["intersection"] = list(tqdm(p.imap(calculate_distance, zip(df1.iterrows(), [df2]*len(df1))), total=len(df1)))
    return df1



In [11]:
df1 = get_intersection_based_on_long_lat(df1, df2)

  0%|          | 0/15000 [00:00<?, ?it/s]Process SpawnPoolWorker-11:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'calculate_distance' on <module '__main__' (built-in)>
Process SpawnPoolWorker-9:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocess

KeyboardInterrupt: 